<a href="https://colab.research.google.com/github/glee1228/net_VLad_keras/blob/master/netVLAD_triplet_keras_ldh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 코랩 연동

In [1]:
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.applications.nasnet import NASNetMobile
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import VGG16
import tensorflow as tf
import keras
from keras import layers

Using TensorFlow backend.


In [28]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab
tb = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://b0eb4050.ngrok.io


In [29]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
    drive.mount('/content/gdrive')
    print('Google Drive is mounted\n')
else:
    print('Google Drive is already mounted\n')

Google Drive is already mounted



In [0]:
tf.set_random_seed(777)

## Data Path

In [0]:
prof_team = False

In [6]:
if prof_team :
  ckpt_path = '/content/gdrive/My Drive/AILeader_Dataset/Checkpoint_jw'
  train_path = './gdrive/My Drive/AILeader_Dataset/train'
  test_path = './gdrive/My Drive/AILeader_Dataset/test'
  ! ls -a ./gdrive/My\ Drive/AILeader_Dataset/train
  
else :
  ckpt_path = './gdrive/My Drive/ckpt'
  train_path = './gdrive/My Drive/train'
  test_path = './gdrive/My Drive/test'
  ! ls -a ./gdrive/My\ Drive/train

 AI	        dorm		   'Front door Sejong'	'Stone statue'
'Clock tower'  'Front door Child'   Museum


In [0]:
os.environ['CUDA_VISIBLE_DEVICES']= '0'

## NetVLAD

In [0]:
# https://github.com/mpkuse/cartwheel_train/blob/master/CustomNets.py

from keras import backend as K
from keras.layers import Layer
import keras
import code
import numpy as np
class NetVLADLayer(Layer):

    def __init__( self, num_clusters, **kwargs ):
        self.num_clusters = num_clusters
        super(NetVLADLayer, self).__init__(**kwargs)

    def build( self, input_shape ):
        self.K = self.num_clusters
        self.D = input_shape[-1]


        self.kernel = self.add_weight( name='kernel',
                                    shape=(1,1,self.D,self.K),
                                    initializer='uniform',
                                    trainable=True )

        self.bias = self.add_weight( name='bias',
                                    shape=(1,1,self.K),
                                    initializer='uniform',
                                    trainable=True )

        self.C = self.add_weight( name='cluster_centers',
                                shape=[1,1,1,self.D,self.K],
                                initializer='uniform',
                                trainable=True)


    def call( self, x ):
        s = K.conv2d( x, self.kernel, padding='same' ) + self.bias
        a = K.softmax( s )
        self.amap = K.argmax( a, -1 )
        a = K.expand_dims( a, -2 ) 
        v = K.expand_dims(x, -1) + self.C 
        v = a * v
        v = K.sum(v, axis=[1, 2])
        v = K.permute_dimensions(v, pattern=[0, 2, 1])

        v = K.l2_normalize( v, axis=-1 )
        v = K.batch_flatten( v )
        v = K.l2_normalize( v, axis=-1 )
        
        # return [v, self.amap]
        return v

    def compute_output_shape( self, input_shape ):
        return (input_shape[0], self.K*self.D )

    def get_config( self ):
        pass
        # base_config = super(NetVLADLayer, self).get_config()
        # return dict(list(base_config.items()))

        # As suggested by: https://github.com/keras-team/keras/issues/4871#issuecomment-269731817
        config = {'num_clusters': self.num_clusters}
        base_config = super(NetVLADLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [0]:
# https://github.com/shamangary/LOUPE_Keras/blob/master/loupe_keras.py
import math
from keras import initializers, layers
class NetVLAD(layers.Layer):
    """Creates a NetVLAD class.
    """
    def __init__(self, feature_size, max_samples, cluster_size, output_dim, **kwargs):
        
        self.feature_size = feature_size
        self.max_samples = max_samples
        self.output_dim = output_dim
        self.cluster_size = cluster_size
        super(NetVLAD, self).__init__(**kwargs)

    def build(self, input_shape):
    # Create a trainable weight variable for this layer.
        self.cluster_weights = self.add_weight(name='kernel_W1',
                                      shape=(self.feature_size, self.cluster_size),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.feature_size)),
                                      trainable=True)
        self.cluster_biases = self.add_weight(name='kernel_B1',
                                      shape=(self.cluster_size,),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.feature_size)),
                                      trainable=True)
        self.cluster_weights2 = self.add_weight(name='kernel_W2',
                                      shape=(1,self.feature_size, self.cluster_size),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.feature_size)),
                                      trainable=True)
        self.hidden1_weights = self.add_weight(name='kernel_H1',
                                      shape=(self.cluster_size*self.feature_size, self.output_dim),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.cluster_size)),
                                      trainable=True)
        
        super(NetVLAD, self).build(input_shape)  # Be sure to call this at the end

    def call(self, reshaped_input):
        activation = K.dot(reshaped_input, self.cluster_weights)
        
        activation += self.cluster_biases
        
        activation = tf.nn.softmax(activation)

        activation = tf.reshape(activation,
                [-1, self.max_samples, self.cluster_size])

        a_sum = tf.reduce_sum(activation,-2,keep_dims=True)
        
        a = tf.multiply(a_sum,self.cluster_weights2)
        
        activation = tf.transpose(activation,perm=[0,2,1])
        
        reshaped_input = tf.reshape(reshaped_input,[-1,
            self.max_samples, self.feature_size])

        vlad = tf.matmul(activation,reshaped_input)
        vlad = tf.transpose(vlad,perm=[0,2,1])
        vlad = tf.subtract(vlad,a)
        vlad = tf.nn.l2_normalize(vlad,1)
        vlad = tf.reshape(vlad,[-1, self.cluster_size*self.feature_size])
        vlad = tf.nn.l2_normalize(vlad,1)
        vlad = K.dot(vlad, self.hidden1_weights)

        return vlad

    def compute_output_shape(self, input_shape):
        return tuple([None, self.output_dim])


## base model 

In [0]:
batch_size= 32
input_shape = (128,128,3)

In [0]:
from keras import Model, Input
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.optimizers import SGD,Adam
from keras.models import Sequential

def initialize_model(backbone= None, input_shape=input_shape, use_pretrained = 'imagenet'):
  base_model = None
  dim = 0
  base_model = backbone(input_shape=input_shape, weights=use_pretrained, include_top= False)
  x = base_model.output
  dim = x[-1].get_shape().as_list()[2]
#   reshaped_input = tf.reshape(x, [-1, features_size])
#   dim = reshaped_input[-1].get_shape().as_list()[0]
  return base_model, dim



In [0]:
## VGG16 or Xception or DenseNet121 or 
## DenseNet169 or DenseNet201 or NASNetMobile or 
## ResNet50 or InceptionResNetV2
model_name = VGG16  

In [13]:
base_model, dim =initialize_model(model_name, input_shape=input_shape)

print('dim : ',dim)

Instructions for updating:
Colocations handled automatically by placer.
dim :  512


## Triplet Sampler

In [0]:
train_triplet_num = 50
test_triplet_num = 10
img_width = 128
img_height = 128

### Train Triplets Pathes

In [15]:
train_dir_pathes = []
i =0
for root,dir,file in os.walk(train_path):
  if i>=1:
    train_dir_pathes.append(root)
  i+=1
train_dir_pathes=sorted(train_dir_pathes)
print(train_dir_pathes) # class dir path

['./gdrive/My Drive/train/AI', './gdrive/My Drive/train/Clock tower', './gdrive/My Drive/train/Front door Child', './gdrive/My Drive/train/Front door Sejong', './gdrive/My Drive/train/Museum', './gdrive/My Drive/train/Stone statue', './gdrive/My Drive/train/dorm']


### Test Triplets Pathes

In [16]:
test_dir_pathes = []
i =0
for root,dir,file in os.walk(test_path):
  if i>=1:
    test_dir_pathes.append(root)
  i+=1
test_dir_pathes=sorted(test_dir_pathes)
print(test_dir_pathes) # class dir path

['./gdrive/My Drive/test/AI', './gdrive/My Drive/test/Clock tower', './gdrive/My Drive/test/Front door Child', './gdrive/My Drive/test/Front door Sejong', './gdrive/My Drive/test/Museum', './gdrive/My Drive/test/Stone statue', './gdrive/My Drive/test/dorm']


In [44]:
import numpy as np
import random
from PIL import Image



def get_triplets(triplet_num,img_width,img_height,dir_pathes):
  if 'train' in str(dir_pathes) :
    print('getting train data..!')
  else :
    print('getting test data..!')
  total_len =len(dir_pathes)*triplet_num
  print('total triplets length : {}'.format(total_len))
  triplets = np.zeros((total_len,3,img_width,img_height,3))

  triplets_len = 0
  for i in range(len(dir_pathes)):
      # set the files of anchor and positive images
      anc_pos_files = [f for f in sorted(os.listdir(dir_pathes[i]))]
      for iteration in range(triplet_num):

          # set the files of negative images
          j = random.choice([num for num in range(len(dir_pathes)) if num not in [i]])
          nega_files = [o for o in sorted(os.listdir(dir_pathes[j]))]

          # get anchor and positive images as numpy array
          pair = np.random.randint(0,len(anc_pos_files),2)
#           print('anchor :',dir_pathes[i],anc_pos_files[pair[0]])
          anchor = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[0]])).resize((img_width,img_height)))
#           print('positive :',dir_pathes[i],anc_pos_files[pair[1]])
          positive = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[1]])).resize((img_width,img_height)))
          
          # get negative images as numpy array
          
          nega_idx = np.random.randint(len(nega_files))
#           print('negative :',dir_pathes[j],nega_files[nega_idx])
          negative = np.array(Image.open('{0}/{1}'.format(dir_pathes[j],nega_files[nega_idx])).resize((img_width,img_height)))
          
          # if image channel !=3 , resample triplets
          if anchor.shape[2]!=3 or positive.shape[2]!=3 or negative.shape[2]!=3:
            # get anchor and positive images as numpy array
            pair = np.random.randint(0,len(anc_pos_files),2)
            anchor = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[0]])).resize((img_width,img_height)))

            positive = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[1]])).resize((img_width,img_height)))

            # get negative images as numpy array
            nega_idx = np.random.randint(len(nega_files))
            negative = np.array(Image.open('{0}/{1}'.format(dir_pathes[j],nega_files[nega_idx])).resize((img_width,img_height)))
          # (optional)visualization
          # plot_triplet(triplet)

          triplets[triplets_len:triplets_len+1,0:1,:,:,:]=anchor/255.
          triplets[triplets_len:triplets_len+1,1:2,:,:,:]=positive/255.
          triplets[triplets_len:triplets_len+1,2:3,:,:,:]=negative/255.

          triplets_len+=1
          if triplets_len%20==0:
            print('{}/{}'.format(triplets_len,total_len))
            
  return triplets

x_train = get_triplets(train_triplet_num,img_width,img_height,train_dir_pathes)
x_test = get_triplets(test_triplet_num,img_width,img_height,test_dir_pathes)


getting train data..!
total triplets length : 350
20/350
40/350
60/350
80/350
100/350
120/350
140/350
160/350
180/350
200/350
220/350
240/350
260/350
280/350
300/350
320/350
340/350
getting test data..!
total triplets length : 70
20/70
40/70
60/70


In [45]:
print(type(x_train))

<class 'numpy.ndarray'>


In [0]:
triplet_dir = './gdrive/My Drive/triplets'
import os
if os.path.exists(triplet_dir)==False:
  os.mkdir(triplet_dir)
else:
  pass

In [50]:
# save triplets as numpy array
train_triplet_path = os.path.join(triplet_dir,'x_train.npy')
test_triplet_path = os.path.join(triplet_dir,'x_test.npy')
print('save file')
np.save(train_triplet_path,x_train)
np.save(test_triplet_path,x_test)

save file


In [0]:
# x_train = np.load(train_triplets_path)

## Loss and Entire Model

In [0]:
##https://codepad.co/snippet/triplet-loss-in-keras-tensorflow-backend
from keras.models import Model
from keras.layers import Input, concatenate,merge
from keras.optimizers import Adam
from keras import optimizers
ALPHA = 0.2  # Triplet Loss Parameter

def triplet_loss(x):
    anchor, positive, negative = x

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), 1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), 1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), ALPHA)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), 0)

    return loss
  
def lossless_triplet_loss(y_true, y_pred, N = 4608, epsilon=1e-8):
    """
    Implementation of the triplet loss function
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    N  --  The number of dimension 
    beta -- The scaling factor, N is recommended
    epsilon -- The Epsilon value to prevent ln(0)
    
    
    Returns:
    loss -- real number, value of the loss
    """
    anchor = y_pred[:,0:N]
    print(anchor.shape)
    positive = y_pred[:,N:N*2]
    print(positive.shape)
    negative = y_pred[:,N*2:N*3]
    print(negative.shape)
    # distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),1)
    
    # distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),1)
    
    #Non Linear Values  
    
    # -ln(-x/N+1)
    pos_dist = -tf.log(-tf.divide((pos_dist),N)+1+epsilon)
    neg_dist = -tf.log(-tf.divide((N-neg_dist),N)+1+epsilon)
    
    # compute loss
    loss = neg_dist + pos_dist
    triplet_loss = tf.reduce_mean(tf.maximum(pos_dist - neg_dist + epsilon, 0.))
    print(loss)
    return triplet_loss
  
def load_model(input_shape,base_model,net_vlad):
    
    anchor = Input(shape=(input_shape[0],input_shape[1],input_shape[2]),name='anchor_input')
    pos = Input(shape=(input_shape[0],input_shape[1],input_shape[2]),name='positive_input')
    neg = Input(shape=(input_shape[0],input_shape[1],input_shape[2]),name='negative_input')
    
    anchor_feature=base_model(anchor)
    pos_feature=base_model(pos)
    neg_feature=base_model(neg)
    
    anchor_vlad = net_vlad(anchor_feature)
    pos_vlad = net_vlad(pos_feature)
    neg_vlad = net_vlad(neg_feature)
    
    
    
#     loss = merge([anchor_vlad, pos_vlad, neg_vlad],mode=triplet_loss, name='loss', output_shape=(1,))
    
#     model = Model(inputs=[anchor_example, positive_example, negative_example],
#                   outputs=loss)
    merged_vector = concatenate([anchor_vlad, pos_vlad, neg_vlad], axis=-1)

    # Define the trainable model
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model = Model(inputs=[anchor, pos, neg], outputs=merged_vector)
#     model.compile(optimizer=Adam(), loss=triplet_loss, metrics=[accuracy])
    model.compile(optimizer=sgd,loss=lossless_triplet_loss)
#     model.compile(loss='mean_absolute_error', optimizer=Adam())
    return model
    

In [0]:
xa=base_model(tf.convert_to_tensor(x_train[0:5,0], np.float32))
xp=base_model(tf.convert_to_tensor(x_train[0:5,1], np.float32))
xn=base_model(tf.convert_to_tensor(x_train[0:5,2], np.float32))

In [0]:
input_shape=(128,128,3)

In [368]:
net_vlad = NetVLADLayer(16)
model = load_model(input_shape,base_model,net_vlad)


(?, ?)
(?, ?)
(?, ?)
Tensor("loss_33/concatenate_47_loss/add_4:0", shape=(?,), dtype=float32)


In [0]:
a=net_vlad(xa)
p=net_vlad(xp)
n=net_vlad(xn)

In [0]:
xe=concatenate([a,p,n],axis=-1)

In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
array = xe.eval(session=sess)

In [373]:
array.shape
print(array[0:4608])

[[ 0.00420239  0.00097386 -0.00864621 ...  0.01454314 -0.01106594
  -0.01862521]
 [ 0.00432085  0.00106746 -0.00864508 ...  0.01444227 -0.01109951
  -0.01861683]
 [ 0.0041827   0.00098127 -0.00864642 ...  0.01432618 -0.01106929
  -0.01862514]
 [ 0.00424047  0.00102774 -0.00862667 ...  0.01468687 -0.01107235
  -0.0186138 ]
 [ 0.00430996  0.00100277 -0.00866738 ...  0.01453572 -0.0110676
  -0.01862215]]


In [374]:
print(array[4608:9216])

[]


In [325]:
pred=lossless_triplet_loss(0,xe)

(5, 4608)
(5, 4608)
(5, 4608)
Tensor("add_47:0", shape=(5,), dtype=float32)


In [326]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    anchor_input[0][0]               
                                                                 positive_input[0][0]             
          

In [0]:
num_epochs = 10

In [0]:
from sklearn.metrics import roc_auc_score

def full_auc(model, ground_truth):
    """
    Measure AUC for model and ground truth on all items.
    Returns:
    - float AUC
    """

    ground_truth = ground_truth.tocsr()

    no_users, no_items = ground_truth.shape

    pid_array = np.arange(no_items, dtype=np.int32)

    scores = []

    for user_id, row in enumerate(ground_truth):

        predictions = predict(model, user_id, pid_array)

        true_pids = row.indices[row.data == 1]

        grnd = np.zeros(no_items, dtype=np.int32)
        grnd[true_pids] = 1

        if len(true_pids):
            scores.append(roc_auc_score(grnd, predictions))

    return sum(scores) / len(scores)

In [0]:
total_len=x_train.shape[0]

In [331]:
# steps_per_epoch * batch size = dataset size
steps_per_epoch = int(total_len/ batch_size)
for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    a=tf.convert_to_tensor(x_train[epoch*batch_size:(epoch+1)*batch_size,0], np.float32)
    p = tf.convert_to_tensor(x_train[epoch*batch_size:(epoch+1)*batch_size,1], np.float32)
    n = tf.convert_to_tensor(x_train[epoch*batch_size:(epoch+1)*batch_size,2], np.float32)

    X = {
        'anchor_input': a,
        'positive_input': p,
        'negative_input': n
    }

    hist= model.fit(X,
              np.ones(a.shape),
              verbose=0,
              steps_per_epoch=steps_per_epoch,
              shuffle=True)
    print(hist.history['loss'])

#     print('AUC %s' % full_auc(model, x_test[epoch]))

Epoch 0
[0.0]
Epoch 1
[0.0]
Epoch 2


KeyboardInterrupt: ignored